#Rewritten for Python 3

In [ ]:
!pip install d2l==0.16.3

In [ ]:
#!pip install -U mxnet-cu101==1.7.0
!pip install mxnet-cu80

In [ ]:
#Refer to Multiple-gpus.ipynb
#https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_computational-performance/multiple-gpus.ipynb#scrollTo=q1afZQLAnY_k

In [ ]:
import mxnet

In [ ]:
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

In [ ]:
%matplotlib inline
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

In [ ]:
# Initialize model parameters
scale = 0.01
W1 = np.random.normal(scale=scale, size=(20, 1, 3, 3))
b1 = np.zeros(20)
W2 = np.random.normal(scale=scale, size=(50, 20, 5, 5))
b2 = np.zeros(50)
W3 = np.random.normal(scale=scale, size=(800, 128))
b3 = np.zeros(128)
W4 = np.random.normal(scale=scale, size=(128, 10))
b4 = np.zeros(10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

# Define the model
def lenet(X, params):
    h1_conv = npx.convolution(data=X, weight=params[0], bias=params[1],
                              kernel=(3, 3), num_filter=20)
    h1_activation = npx.relu(h1_conv)
    h1 = npx.pooling(data=h1_activation, pool_type='avg', kernel=(2, 2),
                     stride=(2, 2))
    h2_conv = npx.convolution(data=h1, weight=params[2], bias=params[3],
                              kernel=(5, 5), num_filter=50)
    h2_activation = npx.relu(h2_conv)
    h2 = npx.pooling(data=h2_activation, pool_type='avg', kernel=(2, 2),
                     stride=(2, 2))
    h2 = h2.reshape(h2.shape[0], -1)
    h3_linear = np.dot(h2, params[4]) + params[5]
    h3 = npx.relu(h3_linear)
    y_hat = np.dot(h3, params[6]) + params[7]
    return y_hat

# Cross-entropy loss function
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
#Data Synchronisation
#For efficient multi-GPU training: 
#2. basic parameters: distribute intput parameters
#3. Ability to sum parameterss across multiple devices: allreduce function

In [ ]:
def get_params(params, device):
    new_params = [p.copyto(device) for p in params]
    for p in new_params:
        p.attach_grad()
    return new_params

In [ ]:
new_params = get_params(params, d2l.try_gpu(0))
print('b1 weight:', new_params[1])
print('b1 grad:', new_params[1].grad)

In [ ]:
def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].copyto(data[0].ctx)
    for i in range(1, len(data)):
        data[0].copyto(data[i])

In [ ]:
data = [np.ones((1, 2), ctx=d2l.try_gpu(i)) * (i + 1) for i in range(2)]
print('before allreduce:\n', data[0], '\n', data[1])
allreduce(data)
print('after allreduce:\n', data[0], '\n', data[1])

In [ ]:
#now testing Deep3D Labs

In [ ]:
#!pip install images2gif

In [ ]:
import mxnet as mx

In [ ]:
!git clone https://github.com/isaacgerg/images2gif.git

In [ ]:
import sys
sys.path
sys.path.append('images2gif/images2gif.py')

In [ ]:
#pip uninstall images2gif

In [ ]:
import images2gif
from images2gif import *

In [ ]:
import numpy as np
import os
import urllib
from urllib import request
import cv2
import matplotlib.pyplot as plt
from PIL import Image
#from images2gif import writeGif
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
if not os.path.exists('deep3d-0050.params'):
    #urllib.request.urlretrieve('http://homes.cs.washington.edu/~jxie/download/deep3d-0050.params', 'deep3d-0050.params')
    #Since the link is deprecated: we use some get it from the deep3d for from another author
    urllib.request.urlretrieve('https://github.com/n1ckfg/deep3d/blob/master/deep3d-0050.params.zip','deep3d.params')

In [ ]:
#unzip the file 
#urllib.request.urlretrieve('https://github.com/n1ckfg/deep3d/blob/master/deep3d-symbol.json','deep3d-symbol.json')
model = mx.model.FeedForward.load('deep3d', 50, mx.gpu(0))

In [ ]:
#Build from source 
#mxnet